Antes de hacer algo asegurate de hacer pull del repo de la clase, crear un nuevo branch, copiar esta carpeta a tu carepta de tareas y trabajar sobre ella.  
Cuando acabes has git add, commit y pull request a los archivos que modificaste.  
Valor de la tarea es de 20 puntos y se entrega el siguiente lunes por la mannana.

Hay un requirement.txt que puedes utilizar.

# Tarea: Registro de Actividades con Decoradores y Clases



Objetivo: Crear un sistema que registre las actividades de una clase Usuario en un archivo de texto.

## Instrucciones:



Define un decorador llamado `registrar_actividad` que, cuando se aplique a un método, registre la actividad en un archivo llamado `registro.txt`. La actividad debe tener el formato: "`[FECHA Y HORA]` - Se ejecutó el método `[NOMBRE DEL MÉTODO]` con los argumentos: `[ARGUMENTOS]`".

Crea una clase llamada Usuario con los siguientes métodos:
        `crear(nombre, apellido)`: establece el nombre y apellido del usuario.
        `obtener_info()`: devuelve el nombre y apellido del usuario.
        `modificar(nombre, apellido)`: modifica el nombre y apellido del usuario.

Asegúrate de aplicar el decorador `registrar_actividad` a estos métodos.

Implementa la lógica para registrar la actividad utilizando el contexto with para manejar el archivo `registro.txt`.

Ejemplo de uso:

```usuario = Usuario()
usuario.crear("Ms", "Kobayashi")
info = usuario.obtener_info()
usuario.modificar("Tohru", "Kobayashi")
```

```
[2023-10-05 10:10:10] - Se ejecutó el método crear con los argumentos: ('Ms', 'Kobayashi')
[2023-10-05 10:10:15] - Se ejecutó el método obtener_info con los argumentos: ()
[2023-10-05 10:10:20] - Se ejecutó el método modificar con los argumentos: ('Tohru', 'Kobayashi')
```


Sugerencias

Sugerencias:

    Utiliza el módulo datetime para obtener la fecha y hora actual cuando registres una actividad.
    En el decorador, usa func.__name__ para obtener el nombre del método que estás registrando.

## Codigo

In [1]:
import datetime

# Decorador para registrar actividad
def registrar_actividad(func):
    def envoltura(*args, **kwargs):
        fecha_hora = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        nombre_metodo = func.__name__
        argumentos = ', '.join([str(a) for a in args[1:]] + [f"{k}={v}" for k, v in kwargs.items()])  # args[1:] para excluir el 'self'
        
        # Utilizando el contexto with para manejar el archivo
        with open('registro.txt', 'a') as archivo:
            archivo.write(f"{fecha_hora} - Se ejecutó el método {nombre_metodo} con los argumentos: {argumentos}\n")
            
        return func(*args, **kwargs)
    return envoltura

# Clase Usuario con métodos decorados
class Usuario:
    def __init__(self):
        self.nombre = ''
        self.apellido = ''

    @registrar_actividad
    def crear(self, nombre, apellido):
        self.nombre = nombre
        self.apellido = apellido

    @registrar_actividad
    def obtener_info(self):
        return self.nombre, self.apellido

    @registrar_actividad
    def modificar(self, nombre, apellido):
        self.nombre = nombre
        self.apellido = apellido

# Ejemplo de uso:
usuario = Usuario()
usuario.crear("Ms", "Kobayashi")
info = usuario.obtener_info()
usuario.modificar("Tohru", "Kobayashi")


# Tarea: Evaluación de Tiempos de Ejecución

Objetivo: Utilizar un decorador para medir y registrar los tiempos de ejecución de diversas operaciones en Python, y guardar los resultados en un archivo llamado tiempos.txt.

## Instrucciones

Define un decorador llamado medir_tiempo que registre el tiempo que tarda en ejecutarse la función decorada. El decorador debe escribir el tiempo de ejecución, junto con el nombre de la función, y el tamano del arreglo, en el archivo tiempos.txt, **separados por comas**. Basicamente un `csv` donde se guarden los tiempos de ejecucion.  

La primera linea del archivo debe de representar el nombre de las columnas, las siguientes lineas representan las observaciones:
`funcion,n,tiempo`.

Ejecuta las funciones con diferentes valores de n = [10, 1000, 10000, 100000, 1000000, 5000000] para generar datos en tiempos.txt.  
Corre cada funcion al menos unas 5 veces para cada tamanno del arreglo. Asegurate de no sobreescribir el archivo txt para que no se elimine la ejecucion.

## Ejemplo de Uso

```
iterar_lista(10000)
iterar_tupla(10000)
iterar_objeto(10000)
usar_yield(10000)

```

```
[2023-10-05 10:10:10] - La función iterar_lista tardó 0.00123 segundos con n=10000
[2023-10-05 10:10:11] - La función iterar_tupla tardó 0.00120 segundos con n=10000
[2023-10-05 10:10:12] - La función iterar_objeto tardó 0.00543 segundos con n=10000
[2023-10-05 10:10:13] - La función usar_yield tardó 0.00093 segundos con n=10000

```

```
tiempos.txt:
funcion,n,tiempo,
asd,1000,10
```

## Codigo

In [7]:
import time
import datetime
import numpy as np
import pandas as pd
# import polars as pl # No se pudo instalar en mi computadora

def medir_tiempo(func):
    def envoltorio(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        duracion = fin - inicio 
        
        tiempo_actual = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{tiempo_actual}] - La función {func.__name__} tardó {duracion:.5f} segundos con n={args[0]}")
        
        with open("tiempos.txt", "a") as f:
            f.write(f"{func.__name__},{args[0]},{duracion:.5f}\n")
        
        return resultado
        
    return envoltorio

# Crear el archivo 'tiempos.txt' con los encabezados si no existe
try:
    with open("tiempos.txt", "x") as f:
        f.write("funcion,n,tiempo\n")
except FileExistsError:
    pass

@medir_tiempo
def sumar_lista(n):
    lista = [i for i in range(n)]
    return sum(lista)

@medir_tiempo
def sumar_tupla(n):
    tupla = tuple(i for i in range(n))
    return sum(tupla)

@medir_tiempo
def sumar_objeto(n):
    class ClaseEjemplo:
        def __init__(self, numero):
            self.numero = numero
            
    objetos = [ClaseEjemplo(i) for i in range(n)]
    return sum(obj.numero for obj in objetos)

@medir_tiempo
def sumar_usando_yield(n):
    def generador():
        for i in range(n):
            yield i
            
    return sum(generador())

@medir_tiempo
def sumar_conjunto(n):
    conjunto = set(range(n))
    return sum(conjunto)

@medir_tiempo
def sumar_valores_diccionario(n):
    diccionario = {i: i for i in range(n)}
    return sum(diccionario.values())

@medir_tiempo
def sumar_numpy_array(n):
    array = np.arange(n)
    return np.sum(array)

@medir_tiempo
def sumar_pandas_series(n):
    serie = pd.Series(range(n))
    return serie.sum()

@medir_tiempo
def sumar_polars_series(n):
    serie = pl.Series("valores", list(range(n)))
    return serie.sum()

In [8]:
for i in range(0,5):
    for n in [10, 1000, 10000, 100000, 1000000, 5000000]:
        sumar_lista(n)
        sumar_tupla(n)
        sumar_objeto(n)
        sumar_usando_yield(n)
        sumar_conjunto(n)
        sumar_valores_diccionario(n)
        sumar_numpy_array(n)
        sumar_pandas_series(n)
        # sumar_polars_series(n)
"/mnt/data/tiempos.txt"

[2023-10-11 17:13:03] - La función sumar_lista tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_tupla tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_objeto tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_usando_yield tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_conjunto tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_valores_diccionario tardó 0.00000 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_numpy_array tardó 0.00301 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_pandas_series tardó 0.00405 segundos con n=10
[2023-10-11 17:13:03] - La función sumar_lista tardó 0.00097 segundos con n=1000
[2023-10-11 17:13:03] - La función sumar_tupla tardó 0.00100 segundos con n=1000
[2023-10-11 17:13:03] - La función sumar_objeto tardó 0.00099 segundos con n=1000
[2023-10-11 17:13:03] - La función sumar_usando_yield tardó 0.00000 segundos con n=10

'/mnt/data/tiempos.txt'

# Tarea Yield

Investiga y explica en tus propias palabras que es un iterador y un el objeto `yield`

Explicacion: